In [10]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob
import csv
import fileinput
from glob import glob
import re
from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from scipy.integrate import trapz, simps

#ej de path=r'C:\Users\Sergio\Desktop\L6y7\18-04-23'
#path='C:/Users/Nicolás Molina/Desktop/L6-7/04-05-23-14kV'
#path=r'C:\Users\Sergio\Desktop\L6y7\04-05-23-19kV'

path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\caracterizacion gan'

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

import matplotlib.ticker as ticker


In [14]:
#plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\caracterizacion gan"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'
plt.close("all")
os.chdir (path)

from matplotlib.ticker import LogLocator, FuncFormatter

## plots crudos
intlamp=np.loadtxt("curva lampara con diodo-corregida.txt",delimiter="	",skiprows=1,unpack=True)
intdiodo=np.loadtxt("curva diodo.txt",delimiter=",",skiprows=1,unpack=True)
transistor=np.loadtxt("respuesta transistor.txt",delimiter="	",skiprows=1,unpack=True)

espectrolamp=np.loadtxt("lampara medida de fabrica.txt",delimiter=",",skiprows=1,unpack=True)

lambdaespec=espectrolamp[0]
irradiazaespeclamp=espectrolamp[1]

lambdalamp=intlamp[0]
escala=intlamp[1]
porcentaje=intlamp[2]
intlamp=escala*porcentaje

lambdadiodo=intdiodo[0]
responsdiodo=intdiodo[1]

lambdagan=transistor[0]
respuestagan=transistor[1]
##########
"""
plt.figure()

plt.plot(lambdalamp,escala*porcentaje) 
plt.title("Respuesta del diodo al setup")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Corriente del diodo (pA)')
plt.legend()
plt.tight_layout()
plt.grid()

plt.figure()
plt.plot(lambdadiodo,responsdiodo)
plt.title("Responsividad del diodo (fabricante)")  
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad (A/W)')
plt.grid()
plt.tight_layout()
plt.legend()

plt.figure()
plt.plot(lambdagan,respuestagan) 
plt.title("Respuesta del transistor al setup (salto de corriente)")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Salto de corriente ($mA$)')
plt.grid()
plt.tight_layout()
plt.legend()
####################################
"""

# Crear un rango común de puntos para interpolar (puedes ajustar este rango según tus datos)
x_common = np.linspace(max(min(lambdalamp), min(lambdadiodo),min(lambdagan)), min(max(lambdalamp), max(lambdadiodo),max(lambdagan)), num=1000)

# Interpolar los datos
f1 = interp1d(lambdalamp, intlamp, kind='linear', fill_value="extrapolate")
f2 = interp1d(lambdadiodo, responsdiodo, kind='linear', fill_value="extrapolate")
f3= interp1d(lambdagan, respuestagan, kind='linear', fill_value="extrapolate")
# Evaluar las funciones interpoladas en los puntos comunes


y1_interp = f1(x_common)
y2_interp = f2(x_common)
y3_interp=f3(x_common)
################################################################################

errorlockin=0.01*y1_interp
errorsmu=0.003 #en mA !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Multiplicar los valores interpolados
potencialamp = y1_interp*10**(-12) *1/ y2_interp  #hago pA / (A/W)


intensidadsetup=potencialamp/0.076

responsividadgan=y3_interp/(1000*intensidadsetup)  #AGREGAR MULTIPLICAR EN LA DIVISION POR EL AREA DEL TRANSISTOR
# Guardar el resultado en un archivo si es necesario

np.savetxt('INTENSIDAD SETUP.txt', np.column_stack((x_common, intensidadsetup)))


errorintensidadsetup=intensidadsetup*np.sqrt((errorlockin/y1_interp)**2)

errorgan=errorsmu

errorresponsividadgan=responsividadgan*np.sqrt((errorintensidadsetup/intensidadsetup)**2+(errorsmu/y3_interp)**2)

##########################################################

# Graficar los resultados

plt.figure()
plt.plot(lambdaespec, irradiazaespeclamp,ls="-",linewidth=2)
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Irradianza espectral ($W/cm^2 \cdot nm$)')
plt.grid()
plt.legend()
plt.title('Irradianza espectral de la lámpara')
plt.tight_layout()


plt.figure()
plt.plot(x_common, y1_interp,ls="-")
plt.fill_between(x_common, y1_interp - errorlockin, y1_interp + errorlockin, color="red", alpha=0.5, label="Error")
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Corriente en el diodo ($pA$)')
plt.grid()
plt.legend()
plt.title('Respuesta del diodo al setup')
plt.tight_layout()


plt.figure()
#plt.plot(lambdadiodo,responsdiodo,label="Respuesta del diodo del fabricante",linewidth=2)
plt.plot(x_common, y2_interp,ls="-",linewidth=2)
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad del diodo ($A/W$)')
plt.grid()
plt.legend()
plt.title('Responsividad del diodo (dato de fábrica)')
plt.tight_layout()

"""
plt.figure()
plt.plot(x_common, potencialamp, linestyle='--')
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Potencia de la lámpara ($W$)')
plt.grid()
plt.title('Potencia de la lámpara medida en el diodo (0.076$mm^2$)')
plt.show()
"""
plt.figure()
plt.plot(x_common,y3_interp) 
plt.fill_between(x_common, y3_interp - errorgan, y3_interp + errorgan, color="red", alpha=0.5, label="Error")
plt.title("Respuesta del transistor al setup")   
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Salto de corriente ($mA$)')
plt.grid()
plt.legend()
plt.tight_layout()

plt.figure()
plt.plot(x_common, intensidadsetup, linestyle='--')
plt.fill_between(x_common, intensidadsetup - errorintensidadsetup, intensidadsetup + errorintensidadsetup, color="red", alpha=0.5, label="Error")
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad de la lámpara ($W/mm^2$)')
plt.grid()
plt.title('Intensidad espectral del setup ')
plt.tight_layout()
plt.show()


plt.figure()
plt.plot(x_common, responsividadgan, linestyle='--')
plt.fill_between(x_common, responsividadgan - errorresponsividadgan, responsividadgan + errorresponsividadgan, color="red", alpha=0.5, label="Error")
plt.legend()
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Responsividad del transistor de GaN-AlGaN ($A\cdot mm^2/W$)')
#plt.yscale("log")
plt.grid()
plt.title('Responsividad del transistor FALTA AREA DEL TRANSIST')
plt.tight_layout()
plt.show()





No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


## a ver si puedo calcular algo

In [95]:
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\films hexagonales"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'
plt.close("all")
os.chdir (path)

plt.close("all")

Efoton=1.2398*10**(-3)*1/x_common   #energia foton en mev
Eprot=10  #energia proton en mev

Ip=(0.1)*10**(-9)   #1nA A=C/s   
qprot=(1.6)*10**(-19)   #en coulombs C
tasaprot=Ip/qprot
Wprot=Eprot*(1.6)*10**(-13)*tasaprot   #pase la energia a Joules  
Ynf=10000    #rendimiento de centelleo en fotones/mev
Yps=1000
Ypvt=2000


d_centellador = 14  # Diámetro del centellador en mm
A_centellador = np.pi * (d_centellador / 2)**2  # Área del centellador en mm2

# Área del detector en cm^2
A_detector = 0.1  # 0.1 mm^2 
eta = A_detector / A_centellador  # Eficiencia de detección

## plots crudos
pl=np.loadtxt("hexagonales pl.txt",delimiter="	",skiprows=4,unpack=True)

titulos=["Ps/Nf 10%","PS","PVT","PVT/$CeF_{3}$ 10%","PVT/$CeF_{3}$ 10% (sep)"]

lambdapl=pl[0]

plnf=pl[1]
plps=pl[2]
plpvt=pl[3]

#PL NORMALIZADO e interpolado
fnf= interp1d(lambdapl, plnf, kind='linear', fill_value="extrapolate")
fpl=interp1d(lambdapl, plps, kind='linear', fill_value="extrapolate")
fpvt=interp1d(lambdapl, plpvt, kind='linear', fill_value="extrapolate")

ynf=fnf(x_common)/max(fnf(x_common))    
yps=fpl(x_common)/max(fpl(x_common))
ypvt=fpvt(x_common)/max(fpvt(x_common))
#plt.figure()
#plt.plot(x_common,ynf)

# integral del pl normalizado
Znf=trapz(ynf,x_common)
Zps=trapz(yps,x_common)
Zpvt=trapz(ypvt,x_common)

nf=ynf/Znf
ps=yps/Zps
pvt=ypvt/Zpvt


integralps= trapz(Yps*Wprot*Efoton*responsividadgan*ps*eta,x_common)
integralnf= trapz(Ynf*Wprot*Efoton*responsividadgan*nf*eta,x_common)
integralpvt=trapz(Ypvt*Wprot*Efoton*responsividadgan*pvt*eta,x_common)

print("salto de corriente ps=",integralps, f"para salto de corriente {Ip*10**(9)} nA")
print("salto de corriente nf=",integralnf, f"para salto de corriente {Ip*10**(9)} nA")
print("salto de corriente pvt=",integralpvt, f"para salto de corriente {Ip*10**(9)} nA")


salto de corriente ps= 0.0001074114695480763 para salto de corriente 0.10000000000000002 nA
salto de corriente nf= 0.001118519610254643 para salto de corriente 0.10000000000000002 nA
salto de corriente pvt= 0.00063764496548888 para salto de corriente 0.10000000000000002 nA
